In [ ]:
!pip install pandas numpy scikit-learn matplotlib seaborn --quiet

In [ ]:
import os
import pandas as pd
import numpy as np
import shutil

In [ ]:


# Create necessary folders inside the Colab environment
os.makedirs('/content/data/raw', exist_ok=True)        # For CERT raw .csv files
os.makedirs('/content/data/processed', exist_ok=True)  # For generated features
os.makedirs('/content/model', exist_ok=True)           # For saving model + scaler

print("✅ Folders created successfully:")
print("• /content/data/raw")
print("• /content/data/processed")
print("• /content/model")

✅ Folders created successfully:
• /content/data/raw
• /content/data/processed
• /content/model


# **LOGON FEATURES**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


file_path = '/content/drive/My Drive/Insider/logon.csv'
logon_df = pd.read_csv(file_path)
logon_df['datetime'] = pd.to_datetime(logon_df['date'], errors='coerce')
logon_df['hour'] = logon_df['datetime'].dt.hour
logon_df['weekday'] = logon_df['datetime'].dt.weekday

#filter relevant activities
logons = logon_df[logon_df['activity'].str.lower() == 'logon']
logoffs = logon_df[logon_df['activity'].str.lower() == 'logoff']



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


"file_path = '/content/drive/My Drive/Insider/logon.csv'\nlogon_df = pd.read_csv(file_path)\nlogon_df['datetime'] = pd.to_datetime(logon_df['date'], errors='coerce')\nlogon_df['hour'] = logon_df['datetime'].dt.hour\nlogon_df['weekday'] = logon_df['datetime'].dt.weekday\n\n#filter relevant activities\nlogons = logon_df[logon_df['activity'].str.lower() == 'logon']\nlogoffs = logon_df[logon_df['activity'].str.lower() == 'logoff']"

In [ ]:
file_path = '/content/drive/My Drive/Insider/psychometric.csv'
email_df = pd.read_csv(file_path)
email_df.head(30)

,employee_name,user_id,O,C,E,A,N
0,Nicholas Fletcher Pruitt,NFP2441,34,39,38,36,21
1,Abraham Dante Rodgers,ADR1517,36,39,13,19,27
2,Medge Wilma Blackburn,MWB4000,27,14,44,22,34
3,Meghan Laurel Salazar,MLS2856,35,49,22,45,28
4,Beau Todd Romero,BTR2026,15,15,41,39,27
5,Zephania Chaim Terry,ZCT1009,35,39,32,14,31
6,Sierra Emily Mendez,SEM1983,42,19,23,31,22
7,Bell Daria White,BDW1459,14,15,21,23,28
8,Fulton August Jackson,FAJ3101,40,50,43,18,28
9,Carl Louis Rice,CLR1239,46,44,38,44,23


FEATURE 1 : Total Logons

In [ ]:
total_logons = logons.groupby('user').size().reset_index(name='total_logons')

FEATURE 2 : After-hours Logons

In [ ]:
logons['after_hours'] = logons['hour'].apply(lambda h: int(h < 8 or h > 18))
after_hours_logons = logons.groupby('user')['after_hours'].sum().reset_index(name='after_hours_logons')

/tmp/ipython-input-6-1967668963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logons['after_hours'] = logons['hour'].apply(lambda h: int(h < 8 or h > 18))


FEATURE 3 : Weekend Logons

In [ ]:
logons['is_weekend'] = logons['weekday'].apply(lambda d: int(d >= 5))
weekend_logons = logons.groupby('user')['is_weekend'].sum().reset_index(name='weekend_logons')

/tmp/ipython-input-7-61957353.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logons['is_weekend'] = logons['weekday'].apply(lambda d: int(d >= 5))


FEATURE 4 : Unique Machines Used

In [ ]:
unique_machines = logons.groupby('user')['pc'].nunique().reset_index(name='unique_machines_used')

FEATURE 5: Average Logon Hour

In [ ]:
avg_logon_hour = logons.groupby('user')['hour'].mean().reset_index(name='avg_logon_hour')

FEATURE 6: Logon Logoff Ratio

In [ ]:
logon_count = logons.groupby('user').size().reset_index(name='logon_count')
logoff_count = logoffs.groupby('user').size().reset_index(name='logoff_count')
log_ratio = pd.merge(logon_count, logoff_count, on='user', how='outer').fillna(0)
log_ratio['logon_logoff_ratio'] = log_ratio['logon_count'] / (log_ratio['logoff_count'] + 1)
logon_logoff_ratio = log_ratio[['user', 'logon_logoff_ratio']]

MERGING ALL LOGON FEATURES

In [ ]:
features_logon = total_logons \
    .merge(after_hours_logons, on='user', how='outer') \
    .merge(weekend_logons, on='user', how='outer') \
    .merge(unique_machines, on='user', how='outer') \
    .merge(avg_logon_hour, on='user', how='outer') \
    .merge(logon_logoff_ratio, on='user', how='outer')

# Fill any missing values
features_logon.fillna(0, inplace=True)

In [ ]:
import shutil

In [ ]:
os.makedirs('/content/drive/MyDrive/insider', exist_ok=True)
features_logon.to_csv('/content/data/processed/features_logon.csv', index=False)

In [ ]:
shutil.copy('/content/data/processed/features_logon.csv', '/content/drive/MyDrive/insider/features_logon.csv')

'/content/drive/MyDrive/insider/features_logon.csv'

In [ ]:
features_logon.head()

,user,total_logons,after_hours_logons,weekend_logons,unique_machines_used,avg_logon_hour,logon_logoff_ratio
0,AAB0162,355,349,0,1,7.016901,0.994398
1,AAB0398,356,356,0,1,6.000000,0.997199
2,AAC0610,665,188,1,1,9.699248,1.713918
3,AAC0668,356,158,0,1,7.556180,0.997199
4,AAC3270,356,0,0,1,8.000000,0.997199


# **FILES FEATURES**

In [ ]:
file_path = '/content/drive/My Drive/Insider/file.csv'
file_df = pd.read_csv(file_path)
file_df['datetime'] = pd.to_datetime(file_df['date'], errors='coerce')

FEATURE 1: USB File Writes

In [ ]:
usb_file_writes = file_df[file_df['to_removable_media'] == True].groupby('user').size().reset_index(name='usb_file_writes')

FEATURE 2: USB File Reads

In [ ]:
usb_file_reads = file_df[file_df['from_removable_media'] == True].groupby('user').size().reset_index(name='usb_file_reads')

FEATURE 3: Unique Files Accessed

In [ ]:
unique_files_accessed = file_df.groupby('user')['filename'].nunique().reset_index(name='unique_files_accessed')

FEATURE 4: File Write Ratio

In [ ]:
total_actions = file_df.groupby('user').size().reset_index(name='total_actions')
write_actions = file_df[file_df['activity'].str.contains('Write', case=False, na=False)].groupby('user').size().reset_index(name='write_actions')
file_write_ratio = pd.merge(total_actions, write_actions, on='user', how='left').fillna(0)
file_write_ratio['file_write_ratio'] = file_write_ratio['write_actions'] / (file_write_ratio['total_actions'] + 1)
file_write_ratio = file_write_ratio[['user', 'file_write_ratio']]

FEATURE 5: Sensitive File Accesses

In [ ]:
keywords = ['secret', 'confidential', 'password', 'private', 'internal']
file_df['is_sensitive'] = file_df['filename'].str.contains('|'.join(keywords), case=False, na=False).astype(int)
sensitive_file_accesses = file_df.groupby('user')['is_sensitive'].sum().reset_index(name='sensitive_file_accesses')

FEATURE 6: Burst File Activity Score

In [ ]:
burst_scores = []
for user, group in file_df.groupby('user'):
    group = group.sort_values('datetime')
    time_diffs = group['datetime'].diff().dt.total_seconds().fillna(99999)  # Large value for first row
    burst_count = (time_diffs <= 300).sum()  # 300 seconds = 5 minutes
    burst_scores.append({'user': user, 'burst_file_activity_score': burst_count})
burst_file_activity_score = pd.DataFrame(burst_scores)

MERGE ALL FILE FEATURES

In [ ]:
features_file = usb_file_writes \
    .merge(usb_file_reads, on='user', how='outer') \
    .merge(unique_files_accessed, on='user', how='outer') \
    .merge(file_write_ratio, on='user', how='outer') \
    .merge(sensitive_file_accesses, on='user', how='outer') \
    .merge(burst_file_activity_score, on='user', how='outer')

# Fill NaNs (users may not have all activity types)
features_file.fillna(0, inplace=True)

In [ ]:
os.makedirs('/content/data/processed', exist_ok=True)
features_file.to_csv('/content/data/processed/features_file.csv', index=False)

# Optionally save to Google Drive (Drive already mounted)
os.makedirs('/content/drive/MyDrive/insider', exist_ok=True)
shutil.copy('/content/data/processed/features_file.csv', '/content/drive/MyDrive/insider/features_file.csv')

'/content/drive/MyDrive/insider/features_file.csv'

In [ ]:
features_file.head()

,user,usb_file_writes,usb_file_reads,unique_files_accessed,file_write_ratio,sensitive_file_accesses,burst_file_activity_score
0,AAB0162,0.0,0.0,8,0.00000,0,3
1,AAB0398,0.0,0.0,7,0.00000,0,5
2,AAC0610,604.0,1056.0,641,0.12689,0,1114
3,AAC0668,0.0,0.0,8,0.00000,0,15
4,AAC3270,0.0,0.0,2,0.00000,0,0


# **EMAILS FEATURES**

In [ ]:
file_path = '/content/drive/My Drive/Insider/email.csv'
email_df = pd.read_csv(file_path)

NameError: name 'pd' is not defined

In [ ]:
email_df['datetime'] = pd.to_datetime(email_df['date'], errors='coerce')
email_df['hour'] = email_df['datetime'].dt.hour

FEATURE 1: Emails Sent

In [ ]:
emails_sent = email_df[email_df['activity'].str.lower() == 'send'].groupby('user').size().reset_index(name='emails_sent')


FEATURE 2: After Hours Emails

In [ ]:
after_hours = email_df[(email_df['activity'].str.lower() == 'send') & ((email_df['hour'] < 8) | (email_df['hour'] > 18))]
after_hours_emails = after_hours.groupby('user').size().reset_index(name='after_hours_emails')


FEATURE 3: Emails With Attachments

In [ ]:
email_df['has_attachment'] = email_df['attachments'].notna().astype(int)
emails_with_attachments = email_df[email_df['activity'].str.lower() == 'send'].groupby('user')['has_attachment'].sum().reset_index(name='emails_with_attachments')

FEATURE 4: Average Email Size

In [ ]:
avg_email_size = email_df[email_df['activity'].str.lower() == 'send'].groupby('user')['size'].mean().reset_index(name='avg_email_size')

FEATURE 5: External Domain Emails

In [ ]:
email_df['to'] = email_df['to'].fillna('')
email_df['external_domain'] = email_df['to'].apply(
    lambda x: int(any('@' in recipient and not recipient.strip().endswith('@dtaa.com')
                      for recipient in str(x).split(';')))
)
external_domain_emails = email_df[email_df['activity'].str.lower() == 'send'].groupby('user')['external_domain'].sum().reset_index(name='external_domain_emails')

FEATURE 6: Burst Email Activity

In [ ]:
burst_scores = []
for user, group in email_df[email_df['activity'].str.lower() == 'send'].groupby('user'):
    group = group.sort_values('datetime')
    time_diffs = group['datetime'].diff().dt.total_seconds().fillna(99999)
    burst_count = (time_diffs <= 300).sum()  # 300 seconds = 5 minutes
    burst_scores.append({'user': user, 'burst_email_activity_score': burst_count})
burst_email_activity_score = pd.DataFrame(burst_scores)

MERGE ALL EMAIL FEATURES

In [ ]:
features_email = emails_sent \
    .merge(after_hours_emails, on='user', how='outer') \
    .merge(emails_with_attachments, on='user', how='outer') \
    .merge(avg_email_size, on='user', how='outer') \
    .merge(external_domain_emails, on='user', how='outer') \
    .merge(burst_email_activity_score, on='user', how='outer')

# Fill missing values (users with no activity will have 0s)
features_email.fillna(0, inplace=True)

In [ ]:
os.makedirs('/content/data/processed', exist_ok=True)
features_email.to_csv('/content/data/processed/features_email.csv', index=False)


os.makedirs('/content/drive/MyDrive/insider', exist_ok=True)
shutil.copy('/content/data/processed/features_email.csv', '/content/drive/MyDrive/insider/features_email.csv')


features_email.head()

,user,emails_sent,after_hours_emails,emails_with_attachments,avg_email_size,external_domain_emails,burst_email_activity_score
0,AAB0162,1122,48.0,123,1.994349e+05,371,253
1,AAB0398,1348,276.0,335,3.568791e+05,678,379
2,AAC0610,388,6.0,224,8.273392e+05,154,34
3,AAC0668,1093,5.0,265,4.068177e+05,497,259
4,AAC3270,120,0.0,94,1.333953e+06,26,0


# **PSYCHOMETRIC FEATURES**

In [ ]:
file_path = '/content/drive/My Drive/Insider/psychometric.csv'
psychometric_df = pd.read_csv(file_path)

In [ ]:
psychometric_df['personality_risk_index'] = (
    psychometric_df['O'] * 0.2 +
    (100 - psychometric_df['C']) * 0.25 +
    psychometric_df['N'] * 0.25 +
    psychometric_df['E'] * 0.15 +
    (100 - psychometric_df['A']) * 0.15
)

In [ ]:
features_psychometric = psychometric_df[['user_id', 'personality_risk_index']]


In [ ]:
os.makedirs('/content/data/processed', exist_ok=True)
features_psychometric.to_csv('/content/data/processed/features_psychometric.csv', index=False)
os.makedirs('/content/drive/MyDrive/insider', exist_ok=True)
shutil.copy('/content/data/processed/features_psychometric.csv', '/content/drive/MyDrive/insider/features_psychometric.csv')

features_psychometric.head()

,user_id,personality_risk_index
0,NFP2441,42.6
1,ADR1517,43.3
2,MWB4000,53.7
3,MLS2856,38.3
4,BTR2026,46.3


# **MERGING ALL FEATURES TO MAKE ONE DATASET**

In [ ]:
logon = pd.read_csv('/content/drive/MyDrive/insider/features_logon.csv')
files = pd.read_csv('/content/drive/MyDrive/insider/features_file.csv')
email = pd.read_csv('/content/drive/MyDrive/insider/features_email.csv')
psychometric = pd.read_csv('/content/drive/MyDrive/insider/features_psychometric.csv')

In [ ]:
merged = logon.merge(files, on='user', how='outer')
merged = merged.merge(email, on='user', how='outer')

In [ ]:
merged = merged.merge(psychometric, left_on='user', right_on='user_id', how='left')

In [ ]:
merged.drop(columns=['user_id'], inplace=True)


In [ ]:
merged.fillna(0, inplace=True)

In [ ]:
merged.to_csv('/content/drive/MyDrive/insider/merged_features.csv', index=False)

merged.head()

,user,total_logons,after_hours_logons,weekend_logons,unique_machines_used,avg_logon_hour,logon_logoff_ratio,usb_file_writes,usb_file_reads,unique_files_accessed,file_write_ratio,sensitive_file_accesses,burst_file_activity_score,emails_sent,after_hours_emails,emails_with_attachments,avg_email_size,external_domain_emails,burst_email_activity_score,personality_risk_index
0,AAB0162,355,349,0,1,7.016901,0.994398,0.0,0.0,8.0,0.00000,0.0,3.0,1122,48.0,123,1.994349e+05,371,253,50.95
1,AAB0398,356,356,0,1,6.000000,0.997199,0.0,0.0,7.0,0.00000,0.0,5.0,1348,276.0,335,3.568791e+05,678,379,40.70
2,AAC0610,665,188,1,1,9.699248,1.713918,604.0,1056.0,641.0,0.12689,0.0,1114.0,388,6.0,224,8.273392e+05,154,34,49.40
3,AAC0668,356,158,0,1,7.556180,0.997199,0.0,0.0,8.0,0.00000,0.0,15.0,1093,5.0,265,4.068177e+05,497,259,44.70
4,AAC3270,356,0,0,1,8.000000,0.997199,0.0,0.0,2.0,0.00000,0.0,0.0,120,0.0,94,1.333953e+06,26,0,43.00


# **MODEL TRAINING**

In [ ]:
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
merged_df = pd.read_csv('/content/drive/MyDrive/insider/merged_features.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
users = merged['user']
X = merged.drop(columns=['user'])

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
iso_forest = IsolationForest(
    n_estimators=100,
    contamination=0.05,
    random_state=42
)
iso_forest.fit(X_scaled)

IsolationForest(contamination=0.05, random_state=42)

In [ ]:
predictions = iso_forest.predict(X_scaled)
anomaly_scores = iso_forest.decision_function(X_scaled)  # Higher = More Normal

In [ ]:
results = pd.DataFrame({
    'user': users,
    'anomaly_score': anomaly_scores,
    'is_anomaly': predictions
})

In [ ]:
results.to_csv('/content/drive/MyDrive/insider/anomaly_predictions.csv', index=False)
print("✅ Anomaly detection completed and predictions saved!")

# ✅ Step 8: Save Model & Scaler
joblib.dump(iso_forest, '/content/drive/MyDrive/insider/isolation_forest_model.pkl')
joblib.dump(scaler, '/content/drive/MyDrive/insider/feature_scaler.pkl')
print("✅ Model and scaler saved successfully!")

# ✅ Optional: Show Results Preview
results.head()

✅ Anomaly detection completed and predictions saved!
✅ Model and scaler saved successfully!


,user,anomaly_score,is_anomaly
0,AAB0162,0.195960,1
1,AAB0398,0.097365,1
2,AAC0610,0.090817,1
3,AAC0668,0.211362,1
4,AAC3270,0.169243,1


In [ ]:
joblib.dump(list(X_train.columns), "model/expected_features.pkl")

NameError: name 'X_train' is not defined